In [41]:
#Effect Modification by Lifestyle Score
library(multcomp)
library(ggpubr)
library(dplyr)
library(mice)
library(miceadds)
library(lme4)
library(ggplot2)
library(rms)
library(mgcv)
library(tidymv)
library("ggthemes")
library(broom)
library(erer)
library(stringr)
library(tidyverse)

#Read the complete set (no imputation)
UKBB_AG2 <- read.delim("~/jupyter/UKBB_AG2_07NOV20.txt", header = TRUE, na.strings=c("",".","NA")) 

#Dichotomize Outcomes for Logistic Regression
#1_CKD
UKBB_AG2$CKD_only.0.0 <- factor(ifelse(UKBB_AG2$CKD.0.0=="CKD controls","CKD controls",
                              ifelse(UKBB_AG2$CKD.0.0=="CKD","CKD",NA)),
                levels = c("CKD controls", "CKD"))
#Set the refernece
UKBB_AG2$CKD_only.0.0 <- relevel(UKBB_AG2$CKD_only.0.0, ref = "CKD controls")

#2_CKD Extreme
UKBB_AG2$CKD_ex.0.0 <- factor(ifelse(UKBB_AG2$CKD.0.0=="CKD controls","CKD controls",
                              ifelse(UKBB_AG2$CKD.0.0=="CKD extreme","CKD extreme",NA)),
                levels = c("CKD controls", "CKD extreme"))
#Set the refernece
UKBB_AG2$CKD_ex.0.0 <- relevel(UKBB_AG2$CKD_ex.0.0, ref = "CKD controls")

#3_Micro
UKBB_AG2$micro.0.0 <- factor(ifelse(UKBB_AG2$ALBUMINERIA.0.0=="micro","micro",
                              ifelse(UKBB_AG2$ALBUMINERIA.0.0=="normo","normo",NA)),
                levels = c("normo", "micro"))
#Set the reference
UKBB_AG2$micro.0.0 <- relevel(UKBB_AG2$micro.0.0, ref = "normo")

#4_Macro
UKBB_AG2$macro.0.0 <- factor(ifelse(UKBB_AG2$ALBUMINERIA.0.0=="macro","macro",
                              ifelse(UKBB_AG2$ALBUMINERIA.0.0=="normo","normo",NA)),
                levels = c("normo", "macro"))
#Set the reference
UKBB_AG2$macro.0.0 <- relevel(UKBB_AG2$macro.0.0, ref = "normo")

#5_Macro
UKBB_AG2$macro.0.0 <- factor(ifelse(UKBB_AG2$ALBUMINERIA.0.0=="macro","macro",
                              ifelse(UKBB_AG2$ALBUMINERIA.0.0=="normo","normo",NA)),
                levels = c("normo", "macro"))

#6_ESKD vs. Macro
UKBB_AG2$ESKD_macro.0.0 <- factor(ifelse(UKBB_AG2$ESKD.0.0=="yes","ESKD",
                              ifelse(UKBB_AG2$ALBUMINERIA.0.0=="macro","macro",NA)),
                levels = c("macro","ESKD"))

#7_DNCKD vs. Control DNCKD
UKBB_AG2$DNCKD2.0.0 <- factor(ifelse(UKBB_AG2$DNCKD.0.0=="yes","DNCKD",
                              ifelse(UKBB_AG2$CTRL_DNCKD.0.0=="yes","DNCKD Control",NA)),
                levels = c("DNCKD Control","DNCKD"))

#8_ESKD vs. Normo, Macro, Micro
UKBB_AG2$ESKD_Albu.0.0 <- factor(ifelse(UKBB_AG2$ESKD.0.0=="yes","ESKD",
                              ifelse(UKBB_AG2$ALBUMINERIA.0.0 %in% c("normo","macro","micro"),"albu",NA)),
                levels = c("albu","ESKD"))

#Set the reference
UKBB_AG2$macro.0.0 <- relevel(UKBB_AG2$macro.0.0, ref = "normo")

#Summarize Counts of Disease Outcomes
table(UKBB_AG2$CKD_only.0.0) #1
table(UKBB_AG2$CKD_ex.0.0) #2
table(UKBB_AG2$micro.0.0) #3
table(UKBB_AG2$macro.0.0) #4
table(UKBB_AG2$ESKD.0.0) #5
table(UKBB_AG2$DN.0.0) #6
table(UKBB_AG2$ALL.0.0) #7
table(UKBB_AG2$ESKD.0.0) #8
table(UKBB_AG2$ESKD_macro.0.0) #8
table(UKBB_AG2$ESKD_Albu.0.0) #9
table(UKBB_AG2$DNCKD2.0.0) #10

#table smoking
table(UKBB_AG2$'SMOKE.0.0')
UKBB_AG2 <- UKBB_AG2 %>% filter(SMOKE.0.0!='Prefer not to answer')
table(UKBB_AG2$'SMOKE.0.0')


CKD controls          CKD 
      349669         6108 


CKD controls  CKD extreme 
      349669          984 


 normo  micro 
348496  14070 


 normo  macro 
348496   1120 


    no    yes 
356332    447 

...


macro  ESKD 
  963   447 


  albu   ESKD 
346560    447 


DNCKD Control         DNCKD 
       326513           645 


             Current                Never Prefer not to answer 
               38166               201909                 1269 
            Previous 
              132143 


             Current                Never Prefer not to answer 
               38166               201909                    0 
            Previous 
              132143 

In [ ]:
UKBB_AG2$SMOKE.0.0 <- relevel(as.factor(SMOKE.0.0),"Never")
UKBB_AG2$GRS_RAW_T2DIR2d2 <- relevel(as.factor(GRS_RAW_T2DIR2d2),"low risk")
M1 <- glm(CKD_only.0.0 ~ SMOKE.0.0*GRS_RAW_T2DIR2d2, data=UKBB_AG2, family=binomial) 

In [ ]:
lsm = lsmeans(M1, ~ SMOKE.0.0*GRS_RAW_T2DIR2d2)

Error in parse(text = x, keep.source = FALSE) : 
  <text>:1:97: unexpected symbol
1: ~relevel(as.factor(SMOKE.0.0), .Av5.Never.Av5.) * relevel(as.factor(GRS_RAW_T2DIR2d2), .Av5.low risk.Av5.
                                                                                                    ^


In [63]:
pairs <- pairs(lsm)
pairs2 <- pairs(lsm,reverse=TRUE)
summary(pairs)
summary(pairs2)

contrast,estimate,SE,df,z.ratio,p.value
Current high risk - Never high risk,-0.20447220,0.15312003,Inf,-1.3353721,9.208552e-01
Current high risk - Previous high risk,-0.48709134,0.15473401,Inf,-3.1479268,4.351649e-02
Current high risk - Current low risk,0.31874517,0.21985214,Inf,1.4498161,8.783106e-01
Current high risk - Never low risk,-0.06976689,0.15447843,Inf,-0.4516287,9.999548e-01
Current high risk - Previous low risk,-0.41733144,0.15562700,Inf,-2.6816133,1.542545e-01
Current high risk - Current medium risk,0.06808646,0.15165902,Inf,0.4489443,9.999568e-01
Current high risk - Never medium risk,-0.15204114,0.14385022,Inf,-1.0569406,9.800542e-01
Current high risk - Previous medium risk,-0.39480776,0.14415255,Inf,-2.7388191,1.343072e-01
Never high risk - Previous high risk,-0.28261914,0.08266469,Inf,-3.4188615,1.821193e-02
Never high risk - Current low risk,0.52321738,0.17670823,Inf,2.9609112,7.499602e-02


contrast,estimate,SE,df,z.ratio,p.value
Never high risk - Current high risk,0.20447220,0.15312003,Inf,1.3353721,9.208552e-01
Previous high risk - Current high risk,0.48709134,0.15473401,Inf,3.1479268,4.351649e-02
Previous high risk - Never high risk,0.28261914,0.08266469,Inf,3.4188615,1.821193e-02
Current low risk - Current high risk,-0.31874517,0.21985214,Inf,-1.4498161,8.783106e-01
Current low risk - Never high risk,-0.52321738,0.17670823,Inf,-2.9609112,7.499602e-02
Current low risk - Previous high risk,-0.80583651,0.17810859,Inf,-4.5244112,2.088616e-04
Never low risk - Current high risk,0.06976689,0.15447843,Inf,0.4516287,9.999548e-01
Never low risk - Never high risk,-0.13470531,0.08218529,Inf,-1.6390440,7.832479e-01
Never low risk - Previous high risk,-0.41732445,0.08515453,Inf,-4.9007896,3.359106e-05
Never low risk - Current low risk,0.38851206,0.17788660,Inf,2.1840435,4.160752e-01


In [ ]:
contr <- rbind(
  "Low Risk in Never Smoker - High Risk in Never Smoker" =  c(0,1,0,0,1,0,0,0,0))
m.pois.contr <- summary(glht(M1, contr))

In [ ]:
m.pois.contr


In [64]:
M1


Call:  glm(formula = CKD_only.0.0 ~ SMOKE.0.0 * GRS_RAW_T2DIR2d2, family = binomial, 
    data = UKBB_AG2)

Coefficients:
                                  (Intercept)  
                                     -4.27778  
                               SMOKE.0.0Never  
                                      0.20447  
                            SMOKE.0.0Previous  
                                      0.48709  
                     GRS_RAW_T2DIR2d2low risk  
                                     -0.31875  
                  GRS_RAW_T2DIR2d2medium risk  
                                     -0.06809  
      SMOKE.0.0Never:GRS_RAW_T2DIR2d2low risk  
                                      0.18404  
   SMOKE.0.0Previous:GRS_RAW_T2DIR2d2low risk  
                                      0.24899  
   SMOKE.0.0Never:GRS_RAW_T2DIR2d2medium risk  
                                      0.01566  
SMOKE.0.0Previous:GRS_RAW_T2DIR2d2medium risk  
                                     -0.02420  

Degrees of F